In [19]:
def distillation(df, column, maxi, mini, first_limit=0, last_limit=-1):
    val_count = df[column].value_counts()
    val_lst = list(pd.DataFrame(val_count).index)
    groups = []  
    
    def single_slice(item,first_limit,last_limit):
        return item[first_limit:last_limit]

    for i in val_lst:    
        if len(i[0].split(',')) == maxi:
            i_splt = i[0].split(',')
            for j in i_splt:
                groups.append([j])
            val_lst.remove(i)
            break

    for i in val_lst:
        if len(i[0].split(',')) <= maxi & len(i[0].split(',')) > mini:
            for j in range(len(i[0].split(','))):
                groups[j].append(i[0].split(',')[j])
        if len(i[0].split(',')) <= mini & len(i[0].split(',')) != 1:
            for j in i[0].split(','):
                for k in range(len(groups)):
                    if j in groups[k]:
                        groups[k].append(j)
        if len(i[0].split(',')) == 1:
            for k in range(len(groups)):
                if single_slice(i[0],first_limit,last_limit) in groups[k]:
                    groups[k].append(i[0])
    return groups

In [24]:
def distillation_plus(df, column, first_limit=0, last_limit=-1):
    val_count = df[column].value_counts()
    val_lst = list(pd.DataFrame(val_count).index)
    maxi = max([len(i) for i in val_lst])
    mini = min([len(i) for i in val_lst])    
    groups = []     
    def single_slice(item,first_limit,last_limit):
        return item[first_limit:last_limit]

    for i in val_lst:    
        if len(i[0].split(',')) == maxi:
            i_splt = i[0].split(',')
            for j in i_splt:
                groups.append([j])
            val_lst.remove(i)
            break

    for i in val_lst:
        if len(i[0].split(',')) <= maxi & len(i[0].split(',')) > mini:
            for j in range(len(i[0].split(','))):
                groups[j].append(i[0].split(',')[j])
        if len(i[0].split(',')) <= mini & len(i[0].split(',')) != 1:
            for j in i[0].split(','):
                for k in range(len(groups)):
                    if j in groups[k]:
                        groups[k].append(j)
        if len(i[0].split(',')) == 1:
            for k in range(len(groups)):
                if single_slice(i[0],first_limit,last_limit) in groups[k]:
                    groups[k].append(i[0])
    return groups

In [25]:
def reorganize_column_plus(df, column, first_limit=0, last_limit=-1):
    distilled_groups = distillation(df, column, first_limit, last_limit)
    column = df[column]
    column_rebuilt = []
    
    from collections import Iterable
    def flatten(lis):
         for item in lis:
                if isinstance(item, Iterable) and not isinstance(item, str):
                    for x in flatten(item):
                        yield x
                else:
                    yield item                
    for i in column:
        if isinstance(i, list):
            if (len(i) == 1) & isinstance(i[0], str) & (',' in i[0]):
                splt_lst = i[0].split(',')
                flat_lst = list(flatten(splt_lst))
            elif (len(i) == 1) & isinstance(i[0], str) & (',' not in i[0]):
                flat_lst = i
            elif (len(i) == 1) & (not isinstance(i[0], str)):
                flat_lst = i
            else:
                flat_lst = list(flatten(i))
            flat_sorted = [np.nan] * len(distilled_groups)
            for x in flat_lst:
                for y in range(len(distilled_groups)):
                    if x in distilled_groups[y]:
                        flat_sorted[y] = x
            column_rebuilt.append(flat_sorted)
        else:
            column_rebuilt.append([i])
            
    return column_rebuilt

In [21]:
def reorganize_column(df, column, maxi, mini, first_limit=0, last_limit=-1):    distilled_groups = distillation(df, column, maxi, mini, first_limit, last_limit)
    column = df[column]
    column_rebuilt = []
    
    from collections import Iterable
    def flatten(lis):
         for item in lis:
                if isinstance(item, Iterable) and not isinstance(item, str):
                    for x in flatten(item):
                        yield x
                else:
                    yield item                
    for i in column:
        if isinstance(i, list):
            if (len(i) == 1) & isinstance(i[0], str) & (',' in i[0]):
                splt_lst = i[0].split(',')
                flat_lst = list(flatten(splt_lst))
            elif (len(i) == 1) & isinstance(i[0], str) & (',' not in i[0]):
                flat_lst = i
            elif (len(i) == 1) & (not isinstance(i[0], str)):
                flat_lst = i
            else:
                flat_lst = list(flatten(i))
            flat_sorted = [np.nan] * len(distilled_groups)
            for x in flat_lst:
                for y in range(len(distilled_groups)):
                    if x in distilled_groups[y]:
                        flat_sorted[y] = x
            column_rebuilt.append(flat_sorted)
        else:
            column_rebuilt.append([i])
            
    return column_rebuilt

In [22]:
# def explosion_extended(df,column,new_column_prefix):
#     df_exploded = df[column].apply(pd.Series)
#     if len(df_exploded.columns) < 2:
#         df_exploded = df_exploded[0].str.extract('(.*),(.*)(,(.*))*')
#     else:
#         for i in df_exploded.columns:
#             df_exploded[i] = df_exploded[i].str.strip()

#     df_exploded = df_exploded.rename(columns = lambda x : new_column_prefix + str(x))
#     df = pd.concat([df[:], df_exploded[:]], axis=1)    
#     return df


In [23]:
def explosion_extended(df,column,new_column_prefix):
    df_strp = df[column].str.strip()
    df_exploded = df_strp.apply(pd.Series)
    if len(df_exploded.columns) < 2:        
        df_exploded = df_exploded[0].str.extract('\'([a-zA-Z0-9- /()_]*)\'')
    else:
        for i in df_exploded.columns:
            df_exploded[i] = df_exploded[i].str.strip()
    df_exploded = df_exploded.rename(columns = lambda x : new_column_prefix + str(x))
    df = pd.concat([df[:], df_exploded[:]], axis=1)    
    return df

In [4]:
def newline_erase(df,column):
    column = df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)[column]
    return column

In [5]:
def explosion(df,column,new_column_prefix):
    df_exploded = df[column].apply(pd.Series)
    df_exploded = df_exploded.rename(columns = lambda x : new_column_prefix + str(x))
    df = pd.concat([df[:], df_exploded[:]], axis=1)    
    return df

In [6]:
def numerization(x):
    return [re.sub('\D', '', i) if i is not None else '' for i in x]

In [26]:
def unification(df, column):
    val_count = df[column].value_counts()
    val_lst = list(pd.DataFrame(val_count).index)
    raw_lst = [val_lst[i].split(',') for i in range(len(val_lst))]
    val_df = pd.DataFrame(raw_lst)
    for i in val_df.columns:
        val_df[i] = val_df[i].str.extract('\'([a-zA-Z0-9- /()_]*)\'')
    raw_lst_cln = [val_df.loc[i].unique() for i in range(val_df.shape[0])]
    total_list=[]
    united = []
    for i in range(val_df.shape[1]):
        total_list.append(val_df[i].unique())
    for i in total_list:
        for j in i:
            united.append(j)
    united_sr = pd.Series(united)
    uniques = united_sr.unique()
    return pd.DataFrame(columns=uniques)

In [2]:
def spread_to_unification(df, column, extension):
    val_spread = df[column]
    val_lst_spread = list(val_spread)
    raw_lst_spread = [i.split(',') if isinstance(i, str) else i for i in val_lst_spread]
    val_df_spread = pd.Series(raw_lst_spread).to_frame()
    val_df_spread = explosion_extended(val_df_spread,0, extension)    
    for i in val_df_spread.columns:
        val_df_spread[i] = val_df_spread[i].str.extract('\'([a-zA-Z0-9- /()_]*)\'')
    
    unified_df = unification(df, column)
    for i in range(val_df_spread.shape[0]):
        row_lst = list(val_df_spread.loc[i][val_df_spread.loc[i].notnull()])
        loc_lst = [True if i in row_lst else False for i in unified_df.columns]
        unified_df = unified_df.append(pd.Series(loc_lst, index=unified_df.columns), ignore_index=True)
        unified_df.loc[~unified_df.any(axis=1)] = np.nan
    return unified_df